In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, rank, dense_rank,row_number
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "id",
"fs.azure.account.oauth2.client.secret": 'secret',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/id/oauth2/token"}


In [ ]:
#dbutils.fs.mount(
#source = "abfss://tokyo-olympicdata@olympicdatastoragetokyo.dfs.core.windows.net/", # contrainer@storageacc
#mount_point = "/mnt/tokyolympic/",
#extra_configs = configs)

In [ ]:
%fs
ls "/mnt/tokyolympic/"

path,name,size,modificationTime
dbfs:/mnt/tokyolympic/raw-data/,raw-data/,0,1717111048000
dbfs:/mnt/tokyolympic/transformed-data/,transformed-data/,0,1717165306000


In [ ]:
#%fs
#mkdirs "/mnt/tokyolympic/transformed-data/" 

res3: Boolean = true

In [ ]:
spark

In [ ]:
athletes = spark.read.format('csv').option("header","true").option("inferSchema","true").load("/mnt/tokyolympic/raw-data/athletes.csv")
coaches = spark.read.format('csv').option("header","true").option("inferSchema","true").load("/mnt/tokyolympic/raw-data/coaches.csv")
entriesgender = spark.read.format('csv').option("header","true").option("inferSchema","true").load("/mnt/tokyolympic/raw-data/entriesgender.csv")
medals = spark.read.format('csv').option("header","true").option("inferSchema","true").load("/mnt/tokyolympic/raw-data/medals.csv")
teams = spark.read.format('csv').option("header","true").option("inferSchema","true").load("/mnt/tokyolympic/raw-data/teams.csv")

In [ ]:
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [ ]:
teams.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [ ]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [ ]:
# Find the top countries with the highest number of gold medals
medals.orderBy("Gold",ascending=False).select("Team_Country","Gold").show()

+--------------------+----+
|        Team_Country|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|                Cuba|   7|
|         New Zealand|   7|
|              Brazil|   7|
|              Canada|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|               Kenya|   4|
|              Poland|   4|
|      Czech Republic|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [ ]:
#top 3 countries with most athletes in a discipline
df=athletes.groupBy(["Country","Discipline"]).count().withColumnRenamed('count', 'athletes_count').orderBy(["Discipline","athletes_count"],ascending=False)

In [ ]:
windowSpec = Window.partitionBy("Discipline").orderBy(col("athletes_count").desc())
df_with_rank = df.withColumn("rank", row_number().over(windowSpec)).filter(col("rank") <= 3)

In [ ]:
df_with_rank.filter(col("Discipline") == "3x3 Basketball").show()

+--------------------+--------------+--------------+----+
|             Country|    Discipline|athletes_count|rank|
+--------------------+--------------+--------------+----+
|               Japan|3x3 Basketball|             8|   1|
|People's Republic...|3x3 Basketball|             8|   2|
|                 ROC|3x3 Basketball|             8|   3|
+--------------------+--------------+--------------+----+



In [ ]:
top_3_df=df_with_rank.groupBy("Discipline").pivot("rank").agg(
    {"Country": "first", "athletes_count": "first"}
)

In [ ]:
top_3_df=top_3_df.select(
    col("Discipline"),
    col("1_first(Country)").alias("First Rank Country"),
    col("1_first(athletes_count)").alias("First Rank Count"),
    col("2_first(Country)").alias("Second Rank Country"),
    col("2_first(athletes_count)").alias("Second Rank Count"),
    col("3_first(Country)").alias("Third Rank Country"),
    col("3_first(athletes_count)").alias("Third Rank Count"))

In [ ]:
top_3_df.show()

+--------------------+--------------------+----------------+--------------------+-----------------+--------------------+----------------+
|          Discipline|  First Rank Country|First Rank Count| Second Rank Country|Second Rank Count|  Third Rank Country|Third Rank Count|
+--------------------+--------------------+----------------+--------------------+-----------------+--------------------+----------------+
|      3x3 Basketball|               Japan|               8|People's Republic...|                8|                 ROC|               8|
|             Archery|United States of ...|               6|               Japan|                6|People's Republic...|               6|
| Artistic Gymnastics|People's Republic...|              12|                 ROC|               12|United States of ...|              11|
|   Artistic Swimming|               Japan|               8|               Italy|                8|               Spain|               8|
|           Athletics|United State

In [ ]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender.withColumn(
    'Avg_Female', entriesgender['Female'] / entriesgender['Total']
).withColumn(
    'Avg_Male', entriesgender['Male'] / entriesgender['Total']
)

In [ ]:
average_entries_by_gender.show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [ ]:
# Find the top countries with the highest number of gold medals
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("Team_Country","Gold").show()

+--------------------+----+
|        Team_Country|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|                Cuba|   7|
|         New Zealand|   7|
|              Brazil|   7|
|              Canada|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|               Kenya|   4|
|              Poland|   4|
|      Czech Republic|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [ ]:
# the original files
athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyolympic/transformed-data/athletes")
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyolympic/transformed-data/coaches")
entriesgender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyolympic/transformed-data/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyolympic/transformed-data/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyolympic/transformed-data/teams")
# the aggregations

#top_gold_medal_countries.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyolympic/transformed-data/aggregations")